# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [2]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


In [15]:
"+".join(list(boston_features.columns))

'CRIM+INDUS+CHAS+RM+AGE+DIS+PTRATIO+B+LSTAT+RAD_(0, 6]+RAD_(6, 24]+TAX_(0, 270]+TAX_(270, 360]+TAX_(360, 712]+target'

In [25]:
boston_target = pd.DataFrame(boston.target, columns=['target'])



## Run an linear model in Statsmodels

In [27]:

X_smaller =boston_features.drop(["RAD_(0, 6]", "TAX_(270, 360]"], axis=1)
X_smaller.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(6, 24]","TAX_(0, 270]","TAX_(360, 712]",target
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,0,0,0,24.0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,0,1,0,21.6
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,0,1,0,34.7
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,0,1,0,33.4
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,0,1,0,36.2


In [30]:
import statsmodels.api as sm
X_int = sm.add_constant(boston_features)
model = sm.OLS(boston_target,X_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.899e+30
Date:                Mon, 06 May 2019   Prob (F-statistic):               0.00
Time:                        00:56:25   Log-Likelihood:                 15270.
No. Observations:                 506   AIC:                        -3.051e+04
Df Residuals:                     492   BIC:                        -3.045e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -6.883e-15   7.91e-15     -0.871      0.384   -2.24e-14    8.65e-15
CRIM            1.599e-14    9.3e-15      1.719      0.086   -2.28e-15    3.43e-14
INDUS          -3.553e-15    1.6e-15     -2.221      0.027    -6.7e-15    -4.1e-16
CHAS           -5.329e-15   3.53e-15     -1.508      0.132   -1.23e-14    1.62e-15
RM              -1.11e-15   1.87e-15     -0.595      0.552   -4.78e-15    2.56e-15
AGE            -6.661e-16   1.54e-15     -0.431      0.666    -3.7e-15    2.37e-15
DIS             2.176e-14   8.39e-15      2.592      0.010    5.27e-15    3.83e-14
PTRATIO         1.776e-15    1.1e-15      1.615      0.107   -3.85e-16    3.94e-15
B                       0    4.4e-15          0      1.000   -8.64e-15    8.64e-15
LSTAT           4.052e-15   1.91e-15      2.120      0.035    2.97e-16    7.81e-15
RAD_(0, 6]     -2.887e-15   3.62e-15     -0.797      0.426      -1e-14    4.23e-15
RAD_(6, 24]    -4.885e-15   4.75e-15     -1.029      0.304   -1.42e-14    4.45e-15
TAX_(0, 270]    -3.22e-15   3.19e-15     -1.011      0.313   -9.48e-15    3.04e-15
TAX_(270, 360] -1.776e-15    3.1e-15     -0.573      0.567   -7.87e-15    4.32e-15
TAX_(360, 712] -1.332e-15   3.01e-15     -0.443      0.658   -7.24e-15    4.58e-15
target             1.0000   1.98e-16   5.06e+15      0.000       1.000       1.000
==============================================================================
Omnibus:                        2.426   Durbin-Watson:                   0.078
Prob(Omnibus):                  0.297   Jarque-Bera (JB):                2.502
Skew:                          -0.161   Prob(JB):                        0.286
Kurtosis:                       2.880   Cond. No.                     1.91e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.79e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [28]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

X_int_sm = sm.add_constant(X_smaller)
model = sm.OLS(boston_target,X_int_sm).fit()
# model.summary()



In [29]:
model.params

const            -1.199041e-14
CRIM             -1.776357e-15
INDUS            -8.881784e-16
CHAS             -5.551115e-15
RM                7.771561e-16
AGE               2.664535e-15
DIS               1.487699e-14
PTRATIO           2.955969e-15
B                -1.110223e-15
LSTAT             1.776357e-15
RAD_(6, 24]      -8.881784e-16
TAX_(0, 270]      8.604228e-16
TAX_(360, 712]    4.329870e-15
target            1.000000e+00
dtype: float64

## Run the same model in Scikit-learn

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

### Statsmodels

### Scikit-learn

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.